In [1]:
import cv2
import numpy as np

In [2]:
#importar las imágenes
img1 = cv2.imread("deforestacion/imgs/1-1999.jpg")
img2 = cv2.imread("deforestacion/imgs/1-2016.jpg")

img3 = cv2.imread("deforestacion/imgs/1-1999.jpg")
img4 = cv2.imread("deforestacion/imgs/1-2016.jpg")

In [3]:
#Convertir la imagen a HSV:
hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)

In [4]:
#Establecemos el rango mínimo y máximo de HSV:
verde_bajos = np.array([35,0,0])
verde_altos = np.array([85, 255, 255])

In [5]:
#Detectamos los píxeles que estén dentro del rango 
#que hemos establecido:
mask1 = cv2.inRange(hsv1, verde_bajos, verde_altos)
mask2 = cv2.inRange(hsv2, verde_bajos, verde_altos)

In [6]:
#Creamos un kernel:
kernel1 = np.ones((6,6),np.uint8) #Matriz de 6x6 llena de '1'
kernel2 = np.ones((6,6),np.uint8) #Matriz de 6x6 llena de '1'

In [7]:
#Eliminamos el ruido con un CLOSE seguido de un OPEN:
mask1 = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, kernel1)
mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, kernel1)

mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernel2)
mask2 = cv2.morphologyEx(mask2, cv2.MORPH_OPEN, kernel2)

In [8]:
#cv2.imshow("mask 1", cv2.resize(mask1, None, fx=0.5, fy=0.5))
#cv2.imshow("maks 2", cv2.resize(mask2, None, fx=0.5, fy=0.5))
#cv2.imshow("mask 1", cv2.resize(img3, None, fx=0.5, fy=0.5))
#cv2.imshow("maks 2", cv2.resize(img4, None, fx=0.5, fy=0.5))

In [9]:
sift = cv2.xfeatures2d.SIFT_create()
kp_1, desc_1 = sift.detectAndCompute(mask1, None)
kp_2, desc_2 = sift.detectAndCompute(mask2, None)
#kp_1, desc_1 = sift.detectAndCompute(img3, None)
#kp_2, desc_2 = sift.detectAndCompute(img4, None)

In [10]:
print("1ra imagen: " + str(len(kp_1)))
print("2da imagen: " + str(len(kp_2)))


#Colores
#4427
#10706

#Blanco y negro
#733
#1674

1ra imagen: 733
2da imagen: 1674


In [11]:
index_params = dict(algorithm = 0, trees=5)
search_params = dict()
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(desc_1, desc_2, k=2)

#Ac[a hay muchos falsos positivos]
#print(len(matches))

In [12]:
good_points = []
for m, n in matches:
    if m.distance < 0.6 * n.distance:
        good_points.append(m)

num_coin = len(good_points)
por_coin = round(100 * num_coin / min(len(kp_1), len(kp_2)), 2)
print("Coincidencias: " + str(num_coin) + 
      "\n --> " + str(por_coin) + "%")

Coincidencias: 49
 --> 6.68%


In [13]:
#cv2.imshow("original", original)

In [14]:
result = cv2.drawMatches(mask1, kp_1, mask2, kp_2, good_points, None)
#result = cv2.drawMatches(img3, kp_1, img4, kp_2, good_points, None)
cv2.imwrite("resultado.jpg", result)
cv2.imshow("result", cv2.resize(result, None, fx=0.5, fy=0.5))
cv2.waitKey(0)
cv2.destroyAllWindows()